In [10]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam,SGD
import numpy as np

import os

from keras.applications.resnet50 import ResNet50


# 本範例不需使用 GPU, 將 GPU 設定為 "無"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [18]:
batch_size = 64 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 2 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 20 # 訓練的 epochs 數量

# create a data generator
datagenForTrain = ImageDataGenerator(rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   channel_shift_range=10,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

datagenForValid = ImageDataGenerator(validation_split=.2)

IMAGE_SIZE = (224, 224)


# load and iterate training dataset
train_it = datagenForTrain.flow_from_directory('../data/kaggle_dogcat/train'
                                               ,target_size=IMAGE_SIZE
                                               , class_mode='categorical'
                                               , batch_size=batch_size
                                               , interpolation='bicubic'
                                               , subset='training')


# load and iterate validation dataset
#valid_it = datagen.flow_from_directory('../data/kaggle_dogcat/train', class_mode='binary', batch_size=batch_size, subset='validation')
valid_it = datagenForValid.flow_from_directory('../data/kaggle_dogcat/train'
                                               ,target_size=IMAGE_SIZE
                                               , class_mode='categorical'
                                               , batch_size=batch_size
                                               , interpolation='bicubic'
                                               , subset='validation')

# confirm the iterator works
batchX = train_it.next()
batchY = valid_it.next()

# load and iterate test dataset
# test_it = datagen.flow_from_directory('../data/kaggle_dogcat/test', class_mode='binary', batch_size=64)

Found 4000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [14]:
# 模型輸出儲存的檔案
WEIGHTS_FINAL = '../model/model-resnet50-final.h5'

NUM_CLASSES=2

# 凍結網路層數
FREEZE_LAYERS = 2

# 輸出各類別的索引值
for cls, idx in train_it.class_indices.items():
    print('Class #{} = {}'.format(idx, cls))

# 以訓練好的 ResNet50 為基礎來建立模型，
# 捨棄 ResNet50 頂層的 fully connected layers
net = ResNet50(include_top=False, weights='imagenet', input_tensor=None,
               input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3))
x = net.output
x = Flatten()(x)

# 增加 DropOut layer
x = Dropout(0.5)(x)

# 增加 Dense layer，以 softmax 產生個類別的機率值
output_layer = Dense(NUM_CLASSES, activation='softmax', name='softmax')(x)

# 設定凍結與要進行訓練的網路層
net_final = Model(inputs=net.input, outputs=output_layer)
for layer in net_final.layers[:FREEZE_LAYERS]:
    layer.trainable = False
for layer in net_final.layers[FREEZE_LAYERS:]:
    layer.trainable = True
    
    
# 使用 Adam optimizer，以較低的 learning rate 進行 fine-tuning
net_final.compile(optimizer=Adam(lr=1e-5),
                  loss='categorical_crossentropy', metrics=['accuracy'])

# 輸出整個網路結構
print(net_final.summary())

Class #0 = cats
Class #1 = dogs
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
_____________________________________________________________________________

In [ ]:
net_final.fit_generator(
    train_it,
    steps_per_epoch = train_it.samples // batch_size,
    validation_data = valid_it, 
    validation_steps = valid_it.samples // batch_size,
    epochs = epochs)

# 儲存訓練好的模型
net_final.save(WEIGHTS_FINAL)



Epoch 1/20
62/62 [==============================] - 2058s 33s/step - loss: 0.8415 - acc: 0.7490 - val_loss: 0.2438 - val_acc: 0.9076
Epoch 2/20


In [ ]:
score = model.evaluate_generator(valid_it,steps=valid_it.samples // batch_size)
print('Test loss:', score[0])
print('Test accuracy:', score[1])